In [1]:
#importing necesary libraries
#This code is using XGboost
from math import sqrt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import xgboost as xgb

In [2]:

data = pd.read_excel("C:/Users/orkun/OneDrive/Masaüstü/Ens 492/Code/Last versions/Simdatawithoutspindlevar.xlsx")

data = data.sample(n=35000, random_state=42)  # Sampling random 35000 simulations for increasing the training speed of ML training

data.head()

,Spindle_Speed,Feed_Rate,Axial_Depth,Radial_Depth,Fx_Max,Fx_Avg,Fy_Max,Fy_Avg,Fz_Max,Fz_Avg
20917,6000,0.02,6.2,4.0,433.815491,229.505084,221.660558,69.372813,80.306306,41.751909
18857,6000,0.21,5.7,2.4,1438.553906,520.110694,258.731918,7.857160,393.031778,138.975724
27507,6000,0.19,7.8,7.6,2187.730041,1608.105426,2232.365811,906.923914,820.961197,543.134115
35527,6000,0.17,9.9,0.8,683.602554,297.031614,236.167651,103.531228,170.085026,73.792460
29950,6000,0.05,8.5,1.2,457.259638,181.536516,253.900156,98.360450,95.488533,37.919533


In [3]:
# Targets and inputs
X = data.drop(columns=['Spindle_Speed', 'Feed_Rate', 'Axial_Depth', 'Radial_Depth'])
y = data[['Feed_Rate', 'Axial_Depth', 'Radial_Depth']]

# Scaling data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_standardized = scaler_X.fit_transform(X)
y_standardized = scaler_y.fit_transform(y)

# Train/Test split %80/%20
X_train, X_test, y_train, y_test = train_test_split(X_standardized, y_standardized, test_size=0.2, random_state=42)


In [4]:
# Creating the XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

# Hyperparameter grid
param_grid = {
    'n_estimators': [5,10,50,100],  # Number of trees
    'max_depth': [3, 5, 7, 10],   # Maximum depth of each tree
    # 'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Learning rate
    # 'subsample': [0.6, 0.8, 1.0],  # Percentage of training samples
    # 'colsample_bytree': [0.6, 0.8, 1.0],  # Percentage of features used in each tree
    'gamma': [0,0.25,0.5],  # Minimum loss reduction
    'min_child_weight': [5,7]  # Minimum sample weight in a leaf node
}

# Grid Search
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Best parameters
best_params = grid_search.best_params_
print("Best parameters:", best_params)

#Pick the best model
best_model = grid_search.best_estimator_


Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters: {'gamma': 0, 'max_depth': 7, 'min_child_weight': 5, 'n_estimators': 50}


In [5]:
# Training set predictions
y_pred = grid_search.predict(X_test)

# Inverse transform the scaled predictions and true values
y_pred_unscaled = scaler_y.inverse_transform(y_pred)
y_test_unscaled = scaler_y.inverse_transform(y_test)

# RMSE Calculation
rmse_feed_rate = np.sqrt(mean_squared_error(y_test_unscaled[:, 0], y_pred_unscaled[:, 0]))
rmse_axial_depth = np.sqrt(mean_squared_error(y_test_unscaled[:, 1], y_pred_unscaled[:, 1]))
rmse_radial_depth = np.sqrt(mean_squared_error(y_test_unscaled[:, 2], y_pred_unscaled[:, 2]))

print(f"Best Parameters: {grid_search.best_params_}")
print(f"RMSE for Feed Rate: {rmse_feed_rate}")
print(f"RMSE for Axial Depth: {rmse_axial_depth}")
print(f"RMSE for Radial Depth: {rmse_radial_depth}")

Best Parameters: {'gamma': 0, 'max_depth': 7, 'min_child_weight': 5, 'n_estimators': 50}
RMSE for Feed Rate: 0.006205949154166953
RMSE for Axial Depth: 0.18631783585572426
RMSE for Radial Depth: 0.14279162629251405


In [6]:


# Adding error margin
error_margin = 0.05
y_pred_lower = y_pred_unscaled * (1 - error_margin)
y_pred_upper = y_pred_unscaled * (1 + error_margin)

# Function to determine given values are valid or not
def is_within_error_margin(y_true, y_pred_lower, y_pred_upper):
    return np.all((y_true >= y_pred_lower) & (y_true <= y_pred_upper), axis=1)

# Accuracy rate calculation
correct_classifications = is_within_error_margin(y_test_unscaled, y_pred_lower, y_pred_upper)
accuracy = np.mean(correct_classifications) * 100

print(f"Correct classification rate(%5): {accuracy:.2f}%")


Correct classification rate(%5): 43.14%


In [7]:

#Adding error margin
error_margin = 0.1
y_pred_lower = y_pred_unscaled * (1 - error_margin)
y_pred_upper = y_pred_unscaled * (1 + error_margin)

# Function to determine given values are valid or not
def is_within_error_margin(y_true, y_pred_lower, y_pred_upper):
    return np.all((y_true >= y_pred_lower) & (y_true <= y_pred_upper), axis=1)

# Accuracy rate calculation
correct_classifications = is_within_error_margin(y_test_unscaled, y_pred_lower, y_pred_upper)
accuracy = np.mean(correct_classifications) * 100

print(f"Correct classification rate(%10): {accuracy:.2f}%")


Correct classification rate(%10): 77.59%
